In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import qutip
import theano
import theano.tensor as T
import QubitNetwork

net = QubitNetwork.QubitNetwork(4, interactions=('all', ['zz']),
                   self_interactions=('all', ['x', 'y']),
                   system_qubits=[0, 1, 2])

In [9]:
net.active_hs

OrderedDict([(0, ['x', 'y']),
             (1, ['x', 'y']),
             (2, ['x', 'y']),
             (3, ['x', 'y'])])

In [10]:
net.active_Js

OrderedDict([((0, 1), ['zz']),
             ((1, 2), ['zz']),
             ((1, 3), ['zz']),
             ((2, 3), ['zz']),
             ((0, 3), ['zz']),
             ((0, 2), ['zz'])])

In [4]:
# J is the theano vector containing all the interactions strengths
J = T.dvector('J')
H = T.tensordot(J, sigma_pairs, axes=1)

expH = T.slinalg.expm(H)

f = theano.function([J], H)
f(get_sigmas_index((0, 1)))
# theano.printing.pydotprint(H, 'testPNG.png')

(16, 8, 8)


array([[ 0.,  0.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [2]:
print('Total number of qubits: {0},\n'
      'Number of ancillary qubits: {1}\n'
      'Number of system qubits: {2}\n'
      'System qubits: {3}'.format(
        net.num_qubits, net.num_ancillas,
        net.num_system_qubits, net.system_qubits))
fredkin_gate = qutip.qip.fredkin().data.toarray()
fredkin_gate = QubitNetwork.complex2bigreal(fredkin_gate)
dataset = net.generate_training_data(fredkin_gate, 1000)
# print(dataset[0][0].shape)
# print(dataset[1][0].shape)
x = T.matrix('x')
y = T.matrix('y')
net.fidelity(x, y)

Total number of qubits: 4,
Number of ancillary qubits: 1
Number of system qubits: 3
System qubits: (0, 1, 2)


mean

In [156]:
net.ancillas_state.reshape(net.ancillas_state.shape[0]).shape

(4,)

In [22]:
# net.hs_factors and net.Js_factors are already in big real matrix form,
# and already multiplied by 1j
H_factors = np.concatenate((net.hs_factors, net.Js_factors), axis=0)
# J is the theano vector containing all the interactions strengths and self energies
J = T.dvector('J')
# this builds the Hamiltonian of the system (in big real matrix form),
# already multiplied with the 1j factor and ready for exponentiation
H = T.tensordot(J, H_factors, axes=1)
# expH is the unitary evolution of the system
expH = T.slinalg.expm(H)

# net.initial_state is the initial state stored as a vector (ket),
# multiplying it on the left by expH amounts to evolve it
expH_times_state = T.dot(expH, net.initial_state)
# build the density matrix out of the evolved state
dm = expH_times_state * expH_times_state.T
dm_real = dm[0:dm.shape[0] // 2, 0:dm.shape[1] // 2]
dm_imag = dm[0:dm.shape[0] // 2, dm.shape[1] // 2:]
# partial trace
num_ancillas = net.num_qubits - len(net.system_qubits)
def col_fn(col_idx, row_idx, matrix):
    subm_dim = 2 ** num_ancillas
    return T.nlinalg.trace(
        matrix[row_idx * subm_dim:(row_idx + 1) * subm_dim, col_idx * subm_dim:(col_idx + 1) * subm_dim])

def row_fn(row_idx, matrix):
    results, _ = theano.scan(fn=col_fn,
                             sequences=T.arange(matrix.shape[1] // 2 ** num_ancillas),
                             non_sequences=[row_idx, matrix])
    return results
dm_real_traced, _ = theano.scan(fn=row_fn,
                             sequences=T.arange(dm_real.shape[0] // 2 ** num_ancillas),
                             non_sequences=[dm_real])
dm_imag_traced, _ = theano.scan(fn=row_fn,
                             sequences=T.arange(dm_imag.shape[0] // 2 ** num_ancillas),
                             non_sequences=[dm_imag])
dm_traced_r1 = T.concatenate((dm_real_traced, -dm_imag_traced), axis=1)
dm_traced_r2 = T.concatenate((dm_imag_traced, dm_real_traced), axis=1)
dm_traced = T.concatenate((dm_traced_r1, dm_traced_r2), axis=0)

target_u = complex2bigreal(qutip.qip.fredkin().data.toarray())
target_evolved_state = T.dot(target_u, net.initial_system_state)
# fidelity = target_evolved_state.T.dot(target_u.dot(target_evolved_state))
fidelity = T.dot(target_evolved_state.T, T.dot(dm_traced, target_evolved_state))


# generate random initial parameters
initial_parameters = np.random.randn(net.num_interactions + net.num_self_interactions)
f = theano.function([J], fidelity)
f(initial_parameters)

AttributeError: 'QubitNetwork' object has no attribute 'initial_state'

In [ ]:
# generate random initial parameters
initial_parameters = np.random.randn(
    net.num_interactions + net.num_self_interactions)
f = theano.function([J], fidelity)
f(initial_parameters)